In [13]:
from utils.stringutils import StringUtils
from utils.logutils import LogUtils
from utils.metafeatures import calculate_metafeatures, MetafeatureMapper

from itertools import cycle, islice
from sklearn import cluster, metrics, manifold, ensemble, model_selection, preprocessing
import pandas as pd
from sklearn import cluster, metrics, manifold, ensemble, model_selection, preprocessing
from preprocess_data import PreprocessedDataset
import numpy as np


In [44]:
# calculate metafeatures
tmp = pd.read_csv('/home/licari/AutoMLExperiments/autocluster/data/type=circles-k=2-n=1000-d=2-noise=0.0.csv')
# Get all column names
columns = tmp.columns

# Assuming the last column is 'y_col' and all others are numeric
file_dict = {
    'numeric_cols': columns[:-1].tolist(),  # All columns except the last one
    'categorical_cols': [],  # Assuming no categorical columns
    'ordinal_cols': [],  # Assuming no ordinal columns
    'y_col': columns[-1]  # The last column
}
# perform outlier detection
preprocess_dict={}
# No need to convert to numpy array here

# Creating PreprocessedDataset object
preprocessed_data = PreprocessedDataset(df=tmp, 
                                        y_col=columns[-1], 
                                        numeric_cols= columns[:-1].tolist(),
                                        categorical_cols=[],
                                        ordinal_cols=[],
                                        ignore_cols=[])


# Now you can get the numpy array if needed
raw_data_np = preprocessed_data.X
isolation_forest_contamination='auto'
predicted_labels = ensemble.IsolationForest(n_estimators=100, 
                                                    warm_start=True,
                                                    # behaviour='new',
                                                    contamination=isolation_forest_contamination).fit_predict(raw_data_np)
idx_np = np.where(predicted_labels == 1)
        
        # remove outliers
raw_data_cleaned = tmp.iloc[idx_np].reset_index(drop=True)
preprocess_dict['df'] = raw_data_cleaned
columns=raw_data_cleaned.columns
prepro_dict = {
    'numeric_cols': columns[:-1].tolist(),  # All columns except the last one
    'categorical_cols': [],  # Assuming no categorical columns
    'ordinal_cols': [],  # Assuming no ordinal columns
    'y_col': columns[-1]  # The last column
}

In [45]:
# Get a list of all available metafeatures
metafeature_list = MetafeatureMapper.getAllMetafeatures()

# Calculate all metafeatures
metafeatures = calculate_metafeatures(raw_data_cleaned, prepro_dict, metafeature_list)

/home/licari/AutoMLExperiments/autocluster/autocluster/utils/metafeatures.py:35: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return np.count_nonzero(X == None) + np.count_nonzero(X == '')
/home/licari/AutoMLExperiments/autocluster/autocluster/utils/metafeatures.py:35: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return np.count_nonzero(X == None) + np.count_nonzero(X == '')
/home/licari/AutoMLExperiments/autocluster/autocluster/utils/metafeatures.py:43: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return (np.count_nonzero(X == '') + np.count_nonzero(X == 0)) / X.size


In [49]:
df1 = pd.read_csv('meta.csv')
df2 = pd.read_csv('prova.csv')


In [50]:
# Filter df2 to have only columns that are in df1
df2_filtered = df2[df1.columns]

# Concatenate df1 with the filtered df2
result_df = pd.concat([df1, df2_filtered], ignore_index=True)


In [53]:
result_df.reset_index(drop=True, inplace=True)
result_df.to_csv('mf.csv', index=False)